# Defining the Fully Convolutional Neural Network

In [2]:
import os
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [3]:
class FCNN(nn.Module):
    
    def __init__(self):
        super(FCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=30, kernel_size=5, stride=2)
        
        self.conv2 = nn.Conv2d(in_channels=30, out_channels=60, kernel_size=3, stride=1)
        self.conv2_drop = nn.Dropout2d()
        
        self.conv3 = nn.Conv2d(in_channels=60, out_channels=60, kernel_size=3, stride=2)
        self.conv3_drop = nn.Dropout2d()
        
        self.conv4 = nn.Conv2d(in_channels=60, out_channels=120, kernel_size=3, stride=1)
        
        self.conv5 = nn.Conv2d(in_channels=120, out_channels=120, kernel_size=3, stride=1)
        
        self.conv5 = nn.Conv2d(in_channels=120, out_channels=1, kernel_size=1, stride=1)
        
        
    def forward(self, data):
        x = F.relu(self.conv1(data))
        x = F.relu(self.conv2_drop(self.conv2(x)))
        x = F.relu(self.conv3_drop(self.conv3(x)))
        x = F.relu(self.conv4(x))
        x = F.sigmoid(self.conv5(x))
        return x
        
network = FCNN()

# Loading the samples

In [4]:
POLLEN_DATA_CREATED = False
cur_dir = os.getcwd()
pollen_files = cur_dir + '/Data/PollenData/'
np_pollen_data = []     # single date will be [img,label]


if not POLLEN_DATA_CREATED:
    for folder in next(os.walk(pollen_files))[1]:
        if folder == 'Real':
            label = 1
        else:
            label = 0
        parent_path = os.path.join(pollen_files, folder)
        for file in os.listdir(parent_path):
            if '.png' in file:
                try:
                    path = os.path.join(parent_path, file)
                    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                    np_pollen_data.append([np.array(img), label])
                except Exception as e:
                    print(folder, file, str(e))

    np.random.shuffle(np_pollen_data)
    print("Data created.")


Data created.


# Data Augmentation
and splitting the dataset into train and validation data. 

In [8]:
#transform to tensor
pollen_data = [[torch.from_numpy(i[0]).view(-1,32,32)/255.0,i[1]] for i in np_pollen_data]

#create datasets
data_amount = len(pollen_data)
train_size = int(data_amount*0.8)
val_size = int(data_amount*0.1)

pollen_training_set = pollen_data[:train_size]
pollen_validation_set = pollen_data[train_size:val_size]
pollen_testing_set = pollen_data[val_size:]

# Train the Network

In [6]:

ep = 10;

#Which optimizer and criterion should we choose? Perform best?
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(network.parameters(), lr=0.001, momentum=0.9)


In [9]:
# from pytorch blitz tutorial

def train(network, trainloader, ep, criterion, optimizer, print_interval):
    for epoch in range(ep):
        
        running_loss = 0.0
    
        for i, (inputs, lables) in enumerate(trainloader):
    
            # zero the parameter gradients
            optimizer.zero_grad()
    
            outputs = network(inputs)
            loss = criterion(outputs, lables)
            loss.backward() #propagate the error back through the network
            optimizer.step() #adjust the weights of the network depending on the propagated error
    
            #that's it.
    
            #Some statistics:
            running_loss += loss.item()
    
            if i % print_interval == print_interval - 1:    # print every x mini-batches
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / print_interval))
                running_loss = 0.0
    
    print('Finished Training')

train(network, pollen_training_set, ep, criterion, optimizer, 32)

RuntimeError: Expected 4-dimensional input for 4-dimensional weight [30, 1, 5, 5], but got 3-dimensional input of size [1, 32, 32] instead

# Validation

In [21]:
def validate(network, testloader):
    
    # We do not need any gradiants here, since we do not train the network.
    # We are only interested in the predictions of the network on the testdata. 
    with torch.no_grad():
        
        for i, (inputs, labels) in enumerate(trainloader):
            outputs = network(inputs) 
            
            predicted = (outputs >= 0.9) # Not 100% sure, if this works...
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))